importing dependencies

In [42]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [43]:
from google.colab import userdata
kaggle_API = userdata.get('kaggle_API')
kaggle_username = "bageshjha"


kaggle_config = {
    "username": kaggle_username,
    "key": kaggle_API
}

In [44]:
!mkdir -p ~/.kaggle
with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_config, f)

!chmod 600 ~/.kaggle/kaggle.json

Data Collection

In [45]:
#!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [46]:
#!unzip imdb-dataset-of-50k-movie-reviews.zip -d ./imdb_data
os.listdir('./imdb_data')

FileNotFoundError: [Errno 2] No such file or directory: './imdb_data'

In [ ]:
!ls

loading the dataset

In [47]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [48]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [49]:
data.shape

(50000, 2)

In [50]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [51]:
 data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [52]:
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

<ipython-input-52-d53fdcf96856>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


In [53]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [54]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


splitting data in train and test

In [55]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [56]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [57]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
tokenizer.fit_on_texts(test_data['review'])
X_train = pad_sequences (tokenizer.texts_to_sequences(train_data['review']),maxlen = 200)
X_test = pad_sequences (tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [65]:
print(X_train.shape)
print(X_train)


(40000, 200)
[[1990    1 1180 ...  206  352 3941]
 [  14    3 1624 ...   89  103    9]
 [   0    0    0 ...    2  712   64]
 ...
 [   0    0    0 ... 1611    2  601]
 [   0    0    0 ...  244  103  125]
 [   0    0    0 ...   70   72 2063]]


In [66]:
print(X_test.shape)
print(X_test)

(10000, 200)
[[   0    0    0 ... 1003  724  157]
 [  12  161   58 ...  383    7    7]
 [   0    0    0 ...   51 1087   97]
 ...
 [   0    0    0 ...  125  199 3252]
 [   0    0    0 ... 1069    1 2334]
 [   0    0    0 ...    1  336   27]]


In [60]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

Building the Model

In [63]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [64]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [67]:
model.build(input_shape=(None, 200))

In [68]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

Training the Model

In [69]:
model.fit(X_train,Y_train, epochs = 5, batch_size = 64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 394ms/step - accuracy: 0.7128 - loss: 0.5456 - val_accuracy: 0.8317 - val_loss: 0.3861
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 413ms/step - accuracy: 0.8541 - loss: 0.3498 - val_accuracy: 0.8580 - val_loss: 0.3310
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 253s 394ms/step - accuracy: 0.8599 - loss: 0.3364 - val_accuracy: 0.8633 - val_loss: 0.3239
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 398ms/step - accuracy: 0.8937 - loss: 0.2688 - val_accuracy: 0.8701 - val_loss: 0.3141
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 403ms/step - accuracy: 0.9129 - loss: 0.2290 - val_accuracy: 0.8795 - val_loss: 0.3168


In [70]:
loss,accuracy = model.evaluate(X_test,Y_test)
print("Test Loss : ",loss)
print("Test Accuracy : ",accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 112ms/step - accuracy: 0.8803 - loss: 0.3050
Test Loss :  0.3017917275428772
Test Accuracy :  0.8838000297546387


Building a predictive system

In [73]:
def predictive_system(input_review):
  input_data = tokenizer.texts_to_sequences([input_review])
  input_data = pad_sequences(input_data, maxlen=200)
  prediction = model.predict(input_data)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment


In [74]:
#example
new_review = "This movie was fantastic! I loved every minute of it."
sentiment = predictive_system(new_review)
print("Sentiment:", sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Sentiment: positive


In [75]:
#example
new_review = "This movie was ass"
sentiment = predictive_system(new_review)
print("Sentiment:", sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Sentiment: negative
